In [ ]:
import math
from sklearn import datasets
import numpy as np

In [ ]:
def entropy(p1,n1):
  if p1==0 and n1==0:
    return 1
  elif p1==0 or n1==0:
    return 0
  else:
    return -(p1/(p1+n1))*math.log2(p1/(p1+n1))-(n1/(p1+n1))*math.log2(n1/(p1+n1))

In [ ]:
def IG(p1,n1,p2,n2):
  num = p1+n1+p2+n2
  num1 = p1+n1
  num2 = p2+n2
  return entropy(p1+p2,n1+n2)-(num1/num*entropy(p1,n1)+num2/num*entropy(p2,n2))
print(IG(21,5,8,30))
print(IG(18,33,11,2))

0.2658748685429416
0.12143188346323375


In [ ]:
data = datasets.load_iris()
feature1 = data.data[0:30][:]
feature2 = data.data[50:80][:]
train_feature = np.concatenate((feature1,feature2),axis=0)
train_target = np.concatenate((data.target[0:30],data.target[50:80]))
feature1 = data.data[30:50][:]
feature2 = data.data[80:100][:]
test_feature = np.concatenate((feature1,feature2),axis=0)
test_target = np.concatenate((data.target[30:50],data.target[80:100]))

def ID3DTtrain(feature,target):
  node = dict()
  node['data'] = range(len(target))
  tree = []
  tree.append(node)
  t = 0
  while t<len(tree):
    idx = tree[t]['data']
    if sum(target[idx])==0:
      tree[t]['leaf'] = 1
      tree[t]['decision'] = 0
    elif sum(target[idx])==len(idx):
      tree[t]['leaf'] = 1
      tree[t]['decision'] = 1
    else:
      bestIG = 0
      for i in range(feature.shape[1]):
        pool = list(set(feature[idx,i]))
        pool.sort()
        for j in range(len(pool)-1):
          thres = (pool[j]+pool[j+1])/2
          G1 = []
          G2 = []
          for k in idx:
            if feature[k,i]<thres:
              G1.append(k)
            else:
              G2.append(k)
          p1 = sum(target[G1]==1)
          n1 = sum(target[G1]==0)
          p2 = sum(target[G2]==1)
          n2 = sum(target[G2]==0)
          thisIG = IG(p1,n1,p2,n2)
          if thisIG>bestIG:
            bestIG = thisIG
            bestG1 = G1
            bestG2 = G2
            bestthres = thres
            bestf = i
      if bestIG > 0:
        tree[t]['leaf'] = 0
        tree[t]['selectf'] = bestf
        tree[t]['threshold'] = bestthres
        tree[t]['child'] = [len(tree),len(tree)+1]
        node = dict()
        node['data'] = bestG1
        tree.append(node)
        node = dict()
        node['data'] = bestG2
        tree.append(node)
      else:
        tree[t]['leaf'] = 1
        tree[t]['decision'] = 1 if sum(target[idx])>len(idx)/2 else 0
    t+=1
  return tree
tree = ID3DTtrain(train_feature,train_target)
print(tree)

[{'data': range(0, 60), 'leaf': 0, 'selectf': 2, 'threshold': np.float64(2.5999999999999996), 'child': [1, 2]}, {'data': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], 'leaf': 1, 'decision': 0}, {'data': [30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59], 'leaf': 1, 'decision': 1}]


In [ ]:
def ID3DTtest(feature, tree):
  [n, f] = feature.shape
  out = np.zeros((n,), dtype=int)
  for i in range(n):
    featurea = feature[i,:]
    t = 0
    while(tree[t]['leaf']==0):
      bestf = tree[t]['selectf']
      bestthres = tree[t]['threshold']
      if featurea[bestf]<bestthres:
        t = tree[t]['child'][0]
      else:
        t = tree[t]['child'][1]
    out[i] = tree[t]['decision']
  return out

In [ ]:
train_prediction = ID3DTtest(train_feature,tree)
print(train_target)
print(train_prediction)
test_prediction = ID3DTtest(test_feature,tree)
print(test_target)
print(test_prediction)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1]


https://zh.wikipedia.org/zh-tw/ROC%E6%9B%B2%E7%BA%BF

 判斷class 1 class 2

In [ ]:
data = datasets.load_iris()
feature1 = data.data[50:80][:]   # class 1 (versicolor)
feature2 = data.data[100:130][:] # class 2 (virginica)
train_feature = np.concatenate((feature1,feature2),axis=0)
train_target = np.concatenate((data.target[50:80],data.target[100:130]))

feature1 = data.data[80:100][:]
feature2 = data.data[130:150][:]
test_feature = np.concatenate((feature1,feature2),axis=0)
test_target = np.concatenate((data.target[80:100],data.target[130:150]))

def ID3DTtrain(feature,target):
  node = dict()
  node['data'] = range(len(target))
  tree = []
  tree.append(node)
  t = 0
  while t<len(tree):
    idx = tree[t]['data']
    if all(target[i]==1 for i in idx):
      tree[t]['leaf'] = 1
      tree[t]['decision'] = 1
    elif all(target[i]==2 for i in idx):
      tree[t]['leaf'] = 1
      tree[t]['decision'] = 2
    else:
      bestIG = 0
      for i in range(feature.shape[1]):
        pool = list(set(feature[idx,i]))
        pool.sort()
        for j in range(len(pool)-1):
          thres = (pool[j]+pool[j+1])/2
          G1 = []
          G2 = []
          for k in idx:
            if feature[k,i]<thres:
              G1.append(k)
            else:
              G2.append(k)
          p1 = sum(target[G1]==2)
          n1 = sum(target[G1]==1)
          p2 = sum(target[G2]==2)
          n2 = sum(target[G2]==1)
          thisIG = IG(p1,n1,p2,n2)
          if thisIG>bestIG:
            bestIG = thisIG
            bestG1 = G1
            bestG2 = G2
            bestthres = thres
            bestf = i
      if bestIG > 0:
        tree[t]['leaf'] = 0
        tree[t]['selectf'] = bestf
        tree[t]['threshold'] = bestthres
        tree[t]['child'] = [len(tree),len(tree)+1]
        node = dict()
        node['data'] = bestG1
        tree.append(node)
        node = dict()
        node['data'] = bestG2
        tree.append(node)
      else:
        tree[t]['leaf'] = 1
        tree[t]['decision'] = 2 if sum(target[idx]==2) > len(idx)/2 else 1
    t+=1
  return tree
tree = ID3DTtrain(feature,target)
print(tree)


[{'data': range(0, 60), 'leaf': 0, 'selectf': 3, 'threshold': np.float64(1.55), 'child': [1, 2]}, {'data': [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 26, 28, 29, 49], 'leaf': 0, 'selectf': 2, 'threshold': np.float64(4.95), 'child': [3, 4]}, {'data': [6, 20, 27, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59], 'leaf': 0, 'selectf': 2, 'threshold': np.float64(5.05), 'child': [5, 6]}, {'data': [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 26, 28, 29], 'leaf': 1, 'decision': 1}, {'data': [49], 'leaf': 1, 'decision': 2}, {'data': [6, 20, 27, 36, 43, 51, 53, 56, 57], 'leaf': 0, 'selectf': 1, 'threshold': np.float64(2.9), 'child': [7, 8]}, {'data': [30, 31, 32, 33, 34, 35, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 50, 52, 54, 55, 58, 59], 'leaf': 1, 'decision': 2}, {'data': [36, 43, 51, 53, 56], 'leaf': 1, 'decision': 2}, {'data': [6, 20,

In [ ]:
train_prediction = ID3DTtest(train_feature,tree)
print(train_target)
print(train_prediction)
test_prediction = ID3DTtest(test_feature,tree)
print(test_target)
print(test_prediction)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2]
[1 1 1 2 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2]


針對testing results 計算TPR、FPR、ACC、PPV、NPV、F1-score  


In [ ]:
# TPR=TP/(TP+FN)
# FPR = FP / N = FP / (FP + TN)
# ACC = (TP + TN) / (P + N)
# PPV = TP / (TP + FP)
# NPV = TN / (TN + FN)
# F1 = 2TP/(P+P')

def compare_result(real_result, test_result):
    TP = TN = FP = FN = 0

    for i in range(len(real_result)):
        r = real_result[i]
        t = test_result[i]

        # 以「2」為正類
        if r == 2 and t == 2:
            TP += 1       # 真實為2，預測為2 → True Positive
        elif r == 1 and t == 1:
            TN += 1       # 真實為1，預測為1 → True Negative
        elif r == 1 and t == 2:
            FP += 1       # 真實為1，預測為2 → False Positive
        elif r == 2 and t == 1:
            FN += 1       # 真實為2，預測為1 → False Negative

    return TP, TN, FP, FN


def calculate_metrics(TP, TN, FP, FN):
    ACC = (TP + TN) / (TP + TN + FP + FN)
    TPR = TP / (TP + FN) if (TP + FN) != 0 else 0
    FPR = FP / (FP + TN) if (FP + TN) != 0 else 0
    PPV = TP / (TP + FP) if (TP + FP) != 0 else 0
    NPV = TN / (TN + FN) if (TN + FN) != 0 else 0
    F1 = 2 * TP / (2 * TP + FP + FN) if (2 * TP + FP + FN) != 0 else 0

    # print(f"TP: {TP}  TN: {TN}  FP: {FP}  FN: {FN}")
    print(f"ACC = {ACC:.3f}")
    print(f"TPR = {TPR:.3f}")
    print(f"FPR = {FPR:.3f}")
    print(f"PPV = {PPV:.3f}")
    print(f"NPV = {NPV:.3f}")
    print(f"F1  = {F1:.3f}")

TP, TN, FP, FN = compare_result(test_target, test_prediction)
calculate_metrics(TP, TN, FP, FN)

print('TP: '+str(TP)+' TN: '+str(TN)+' FP: '+str(FP)+' FN: '+str(FN))

ACC = 0.950
TPR = 1.000
FPR = 0.100
PPV = 0.909
NPV = 1.000
F1  = 0.952
TP: 20 TN: 18 FP: 2 FN: 0
